# Face Recognition APP

Install usefull libraries : 
1. Labelme : Application pour permettre l'annotation d'images
2. Open CV
3. Albumentations : Bibliothèque pour l'augmentation des données dans la vision par ordinateur.

In [ ]:
!pip install opencv-python tensorflow labelme albumentations matplotlib 

In [207]:
import os 
import time
import cv2
import uuid

## 1. Récupération des données 

**Code utile pour récupérer les données de la web cam utilisant open cv**

In [208]:
IMAGES_PATH = os.path.join('data','images')
number_images = 30

In [209]:
cap = cv2.VideoCapture(0)
for imgnum in range(number_images):
    print('Collecting image {}'.format(imgnum))
    ret, frame = cap.read()
    imgname = os.path.join(IMAGES_PATH,f'{str(uuid.uuid1())}.jpg')
    cv2.imwrite(imgname, frame)
    cv2.imshow('frame', frame)
    time.sleep(0.5)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [210]:
# !labelme # Ouvrir l'application label me, ne pas oublier de changer le repertoire pour les outputs 


## 2. Revu du jeu de données et construction d'une fonction pour charger les images 


In [211]:
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt 

In [212]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus : 
    tf.config.experimental.set_memory_growth(gpu,True)

### Charger les données en un tf data pipeline

In [213]:
# Créer un jeu de données en spécifiant le dossier comportant nos images
images = tf.data.Dataset.list_files('data\\images\\*.jpg', shuffle = True)

InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: data\\images\\*.jpg'

In [ ]:
# Permet de parcourir le tenseur images
images.as_numpy_iterator().next()

In [ ]:
def load_image(x) : 
    byte_img = tf.io.read_file(x) #io pour input/output, module pour écrire, lire  ou charger fichier. byte_img est un tenseur d'octet
    img = tf.io.decode_jpeg(byte_img) #décoder la suite d'octet en une image 
    return img
    

In [ ]:
images = images.map(load_image) # Permet de passer la fonction sur toutes les images 

In [ ]:
images.as_numpy_iterator().next() 

### Visualiser nos images

In [ ]:
images_generator = images.batch(4).as_numpy_iterator() #batch permet de prendre plusieurs données en même temps

In [ ]:
plot_images = images_generator.next()

In [ ]:
fig, ax = plt.subplots(ncols = 4, figsize = (28,20)) # Créer une figure avec un tableau d'axes de 4 colonnes
for idx, image in enumerate(plot_images)  : 
    ax[idx].imshow(image) # Place l'image 1 sur l'axe 0 etc...
plt.show()
    

## 3. Séparer les données en train / test / val

In [ ]:
# 90 * 0.7 pour le train set = 63
# 90 * 0.15 pour le test et val = 14 et 13

Une fois que l'on a bougé les images dans les dossiers correspondants, on doit également bouger les labels associés

In [ ]:
# Ce code permet de changer les labels du dossiers data\label aux nouveaux dossiers correspondants
for folder in ['train','test','val'] : 
    for file in os.listdir(os.path.join('data',folder,'images')) : 
        filename = file.split('.')[0]+'.json' 
        existing_filepath = os.path.join('data','labels', filename)
        if os.path.exists(existing_filepath): 
            new_filepath = os.path.join('data',folder,'labels',filename)
            os.replace(existing_filepath, new_filepath)  

# 4. Augmenter les images et les labels avec Albumentations

In [ ]:
import albumentations as alb

Pour augmenter nos images et labels, on utilise la librairie albumentations. Tout d'abord on crée une pipeline des transformations que l'on veut effectuer avec l'objet 'alb.Compose' en faisant passer une liste de toutes les transformations que l'on désire

In [ ]:
augmentor = alb.Compose([alb.RandomCrop(width=450, height=450), 
                         alb.HorizontalFlip(p=0.5), 
                         alb.RandomBrightnessContrast(p=0.2),
                         alb.RandomGamma(p=0.2), 
                         alb.RGBShift(p=0.2), 
                         alb.VerticalFlip(p=0.5)], 
                       bbox_params=alb.BboxParams(format='albumentations', 
                                                  label_fields=['class_labels']))

Petit test sur une image du train set 

In [ ]:
img = cv2.imread(os.path.join('data','train','images','582357ab-5040-11ef-b567-8b544a84180a.jpg'))

In [ ]:
# Comme vu sur coursera , on charge un fichier json et 'r' pour lire le fichier
with open(os.path.join('data','train','labels','582357ab-5040-11ef-b567-8b544a84180a.json'),'r') as l : 
    label = json.load(l)

In [ ]:
label

On récupère maintenant les coordonnées du label sur une seule liste

In [ ]:
label['shapes'][0]['points']

In [ ]:
label['shapes'][0]['points'][0][0]

In [ ]:
coord = [0,0,0,0]
coord[0] = label['shapes'][0]['points'][0][0]
coord[1] = label['shapes'][0]['points'][0][1]
coord[2] = label['shapes'][0]['points'][1][0]
coord[3] = label['shapes'][0]['points'][1][1]

In [ ]:
coord


On normalise maintenant nos coordonnées 


In [ ]:
coord_norm = list(np.divide(coord, [640,480,640,480]))

**On peut désormais regarder nos augmentations sur l'image et son label associé**

In [ ]:
augmented = augmentor(image=img, bboxes=[coord_norm], class_labels=['face'])

In [ ]:
augmented.keys()

In [ ]:
cv2.rectangle(augmented['image'], 
              tuple(np.multiply(augmented['bboxes'][0][:2], [450,450]).astype(int)),
              tuple(np.multiply(augmented['bboxes'][0][2:], [450,450]).astype(int)), 
                    (255,0,0), 2)

plt.imshow(augmented['image'])

## 5. Créer la pipeline d'augmentations des données sur toutes nos images / labels 

In [ ]:
for partition in ['train','test','val']: 
    for image in os.listdir(os.path.join('data', partition, 'images')):
        img = cv2.imread(os.path.join('data', partition, 'images', image))

        coords = [0,0,0.00001,0.00001]
        label_path = os.path.join('data', partition, 'labels', f'{image.split(".")[0]}.json')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)

            coords[0] = label['shapes'][0]['points'][0][0]
            coords[1] = label['shapes'][0]['points'][0][1]
            coords[2] = label['shapes'][0]['points'][1][0]
            coords[3] = label['shapes'][0]['points'][1][1]
            coords = list(np.divide(coords, [640,480,640,480]))

        try: 
            for x in range(60):
                augmented = augmentor(image=img, bboxes=[coords], class_labels=['face'])
                cv2.imwrite(os.path.join('aug_data', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

                annotation = {}
                annotation['image'] = image

                if os.path.exists(label_path):
                    if len(augmented['bboxes']) == 0: 
                        annotation['bbox'] = [0,0,0,0]
                        annotation['class'] = 0 
                    else: 
                        annotation['bbox'] = augmented['bboxes'][0]
                        annotation['class'] = 1
                else: 
                    annotation['bbox'] = [0,0,0,0]
                    annotation['class'] = 0 


                with open(os.path.join('aug_data', partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)

**Maintenant il faut créer les datasets train / test / val avec les nouvelles images augmentées**

In [ ]:
train_images = tf.data.Dataset.list_files('aug_data\\train\\images\\*.jpg', shuffle = False)
train_images = train_images.map(load_image)
train_images = train_images.map(lambda x: tf.image.resize(x, (120,120))) #Pour améliorer notre réseau de neuronnes
train_images = train_images.map(lambda x : x/255) #Normaliser les images 

In [ ]:
test_images = tf.data.Dataset.list_files('aug_data\\test\\images\\*.jpg', shuffle = False)
test_images = test_images.map(load_image)
test_images = test_images.map(lambda x: tf.image.resize(x, (120,120)))
test_images = test_images.map(lambda x : x/255)

In [ ]:
val_images = tf.data.Dataset.list_files('aug_data\\val\\images\\*.jpg', shuffle = False)
val_images = val_images.map(load_image)
val_images = val_images.map(lambda x: tf.image.resize(x, (120,120)))
val_images = val_images.map(lambda x : x/255)

## 6. Préparer les labels

In [ ]:
def load_labels(label_path):
    with open(label_path.numpy(), 'r', encoding = "utf-8") as f: #convertit le tenseur en chaine de carac
        label = json.load(f)
        
    return [label['class']], label['bbox']

In [ ]:
train_labels = tf.data.Dataset.list_files('aug_data\\train\\labels\\*.json', shuffle=False)
train_labels = train_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16])) # py_function pour utiliser des fonctions pythons arbitraires

In [ ]:
test_labels = tf.data.Dataset.list_files('aug_data\\test\\labels\\*.json', shuffle=False)
test_labels = test_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
val_labels = tf.data.Dataset.list_files('aug_data\\val\\labels\\*.json', shuffle=False)
val_labels = val_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
train_labels.as_numpy_iterator().next()

## 7. Combiner l'image et le label en 1 échantillon

In [ ]:
print(len(train_images), len(test_images), len(val_images), len(train_labels), len(test_labels), len(val_labels))

**On crée maintenant nos datasets finaux composés des images et de leurs labels associés**

In [ ]:
train = tf.data.Dataset.zip((train_images, train_labels)) # Combine les images et les labels en 1 tuple
train = train.shuffle(5000) # Représente le buffer de mélange (doit être >len(train_images) pour éviter la généralisation
train = train.batch(8) # regroupe en des lots de 8 
train = train.prefetch(4) # Permet de pré charger 4 lots en mémoire, ce qui améliore l'efficacité de l'entraînement.

In [ ]:
train_images

In [ ]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)

# Assure-toi que les formes sont correctes
print(train_images.shape)  # Par exemple, (num_samples, height, width, channels)
print(train_labels.shape)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))

# Shuffle, batch, et prefetch
train_dataset = train_dataset.shuffle(buffer_size=5000)  # Le buffer_size doit être >= len(train_images)
train_dataset = train_dataset.batch(8)
train_dataset = train_dataset.prefetch(buffer_size=4) 

In [ ]:
test = tf.data.Dataset.zip((test_images, test_labels))
test = test.shuffle(1300)
test = test.batch(8)
test = test.prefetch(4)

In [ ]:
val = tf.data.Dataset.zip((val_images, val_labels))
val = val.shuffle(1000)
val = val.batch(8)
val = val.prefetch(4)

In [ ]:
data_samples = train.as_numpy_iterator()


In [ ]:
res = data_samples.next()


**On peut afficher nos images avec les cadres associés** 

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = res[0][idx].copy()
    sample_coords = res[1][1][idx]
    
    cv2.rectangle(sample_image, 
                  tuple(np.multiply(sample_coords[:2], [120,120]).astype(int)),
                  tuple(np.multiply(sample_coords[2:], [120,120]).astype(int)), 
                        (255,0,0), 2)

    ax[idx].imshow(sample_image)

## 8. Construire le model de Deep Learning

On importe d'abord les modules nécessaires de l'API Keras 


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, Add, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

In [ ]:
vgg = VGG16(include_top = False) # Include top False car on veut personnaliser le model
# préfabriqué avec notre problème de classification / regression

In [ ]:
vgg.summary() # commande pour voir l'architecture du modèle

**On peut maintenant crée le modèle qui nous permettra de faire notre face recognition, et on commence par l architecture du modèle**

In [ ]:
def build_model(): 
    # On crée la couche d'entrée qui est une image rgb de 120*120
    input_layer = Input(shape=(120,120,3))
    # On utilise le réseau pré entrainé vgg16
    vgg = VGG16(include_top=False)(input_layer)
    # Maintenant il faut séparer notre problème en deux sous problèmes : 
    # Classification (face ou pas) et régression (estimé les positions du cadre)
    
    # Classification Model
    # On ajoute un globalmaxpooling (couramment utilisé à la fin du CNN
    # Pour réduire rapidement les dimensions de l'image
    f1 = GlobalMaxPooling2D()(vgg)
    class1 = Dense(2048, activation='relu')(f1)
    # Sigmoid car c'est une classification binaire (0 ou 1)
    class2 = Dense(1, activation='sigmoid')(class1)
    
    # Bounding box model
    f2 = GlobalMaxPooling2D()(vgg)
    regress1 = Dense(2048, activation='relu')(f2)
    # 4 unité car on a 4 coordonnées pour le cadre 
    regress2 = Dense(4, activation='sigmoid')(regress1)
    # On crée le modèle final avec le module Model(input, output)
    facetracker = Model(inputs=input_layer, outputs=[class2, regress2])
    return facetracker

In [ ]:
facetracker = build_model()

In [ ]:
facetracker.summary()

In [ ]:
X,y = train.as_numpy_iterator().next()

In [ ]:
y[0].shape

In [ ]:
classes, coords = facetracker.predict(X)

In [ ]:
classes, coords

## 9. Définir le coût et la fonction d'optimisation


In [ ]:
BATCH = len(train)
lr_decay = (1./0.75 -1) / BATCH # Pour baisser le learning rate progressivement afin de converger plus vite 

In [ ]:
# On Crée maintenant notre fonction d'optimisation 
opt = tf.keras.optimizers.Adam(learning_rate = 0.0001)

In [ ]:
# Loss pour notre régression pour les cadres 
def localization_loss(y_true, yhat):            
    delta_coord = tf.reduce_sum(tf.square(y_true[:,:2] - yhat[:,:2]))
                  
    h_true = y_true[:,3] - y_true[:,1] 
    w_true = y_true[:,2] - y_true[:,0] 

    h_pred = yhat[:,3] - yhat[:,1] 
    w_pred = yhat[:,2] - yhat[:,0] 
    
    delta_size = tf.reduce_sum(tf.square(w_true - w_pred) + tf.square(h_true-h_pred))
    
    return delta_coord + delta_size

In [ ]:
classloss = tf.keras.losses.BinaryCrossentropy() #classification binaire
regressloss = localization_loss

In [ ]:
localization_loss(y[1], coords)


In [ ]:
classloss(y[0], classes)


In [ ]:
regressloss(y[1], coords)


In [232]:
class FaceTracker(Model): 
    def __init__(self, facetracker,  **kwargs): 
        super().__init__(**kwargs)
        self.model = facetracker

    def compile(self, opt, classloss, localizationloss, **kwargs):
        super().compile(**kwargs)
        self.closs = classloss
        self.lloss = localizationloss
        self.opt = opt
    
    def train_step(self, batch, **kwargs): 
        print(batch)
        X, y = batch
        print(X, y)
        
        with tf.GradientTape() as tape: 
            classes, coords = self.model(X, training=True)
            print("Classes shape:", classes.shape)
            print("Coords shape:", coords.shape)
            print("y[0] shape (class labels):", y[0].shape)
            print("y[1] shape (bounding boxes):", y[1].shape)

            
            batch_classloss = self.closs(y[0], classes)
            batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
            
            total_loss = batch_localizationloss+0.5*batch_classloss
            
            grad = tape.gradient(total_loss, self.model.trainable_variables)
        
        opt.apply_gradients(zip(grad, self.model.trainable_variables))
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
    
    def test_step(self, batch, **kwargs): 
        X, y = batch
        
        classes, coords = self.model(X, training=False)
        
        batch_classloss = self.closs(y[0], classes)
        batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
        total_loss = batch_localizationloss+0.5*batch_classloss
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
        
    def call(self, X, **kwargs): 
        return self.model(X, **kwargs)

In [233]:
model = FaceTracker(facetracker)


In [234]:
model.compile(opt, classloss, regressloss)


In [235]:
logdir='logs'


In [236]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)


In [237]:
hist = model.fit(train, epochs=40, validation_data=val, callbacks=[tensorboard_callback])


Epoch 1/40
(<tf.Tensor 'data:0' shape=(None, 120, 120, None) dtype=float32>, (<tf.Tensor 'data_1:0' shape=<unknown> dtype=uint8>, <tf.Tensor 'data_2:0' shape=<unknown> dtype=float16>))
Tensor("data:0", shape=(None, 120, 120, None), dtype=float32) (<tf.Tensor 'data_1:0' shape=<unknown> dtype=uint8>, <tf.Tensor 'data_2:0' shape=<unknown> dtype=float16>)
Classes shape: (None, 1)
Coords shape: (None, 4)
y[0] shape (class labels): <unknown>
y[1] shape (bounding boxes): <unknown>


ValueError: Cannot take the length of shape with unknown rank.

In [216]:
for batch in train.take(1):
    images, (class_labels, bbox_coords) = batch
    print("Images shape:", images.shape)
    print("Class labels shape:", class_labels.shape)
    print("Bounding box coordinates shape:", bbox_coords.shape)

Images shape: (8, 120, 120, 3)
Class labels shape: (8, 1)
Bounding box coordinates shape: (8, 4)
